In [ ]:
import sys
import os
import time
import pandas as pd

# Add the 'src' directory to the path so we can import market_feed
# Adjust ".." if your notebook is in a different folder relative to 'src'
sys.path.append(os.path.abspath("../src"))

from market_feed.manager import FeedManager

print("Libraries imported successfully.")

In [ ]:
# specific config to test Bloomberg (SPX US Equity)
test_config = [
    {
        "tab_name": "SPY",
        "base_symbol": "SPY",    # Will map to SPX US Equity
        "settlement": "usd",
        "source": "bloomberg"    # <--- This forces the BloombergAdapter
    }
]

print("Configuration defined.")

In [ ]:
# Initialize Manager with our test config
# We pass empty api_keys because Bloomberg Desktop API doesn't need them
manager = FeedManager(config_path=None, keys_path=None, api_keys={})

# Manually inject the config since we didn't load from file
manager._market_config = test_config
manager._init_adapters() 

# DIAGNOSTIC CHECK
bbg_adapter = manager.adapters.get('bloomberg')

if bbg_adapter:
    print("✅ Bloomberg Adapter successfully loaded.")
else:
    print("❌ Bloomberg Adapter NOT loaded.")
    print("Reason: 'blpapi' library might be missing or failed to import.")
    print("Try: pip install --index-url=https://bcms.bloomberg.com/pip/simple blpapi")

In [ ]:
if bbg_adapter:
    print("Attempting to connect to Bloomberg Terminal...")
    bbg_adapter.start()
    
    # Wait a few seconds for session connection
    time.sleep(3)
    
    if bbg_adapter.connected:
        print("✅ Connected to Bloomberg Terminal (Port 8194)!")
    else:
        print("❌ Connection Failed.")
        print("Check: Is the Bloomberg Terminal application logged in and running?")
else:
    print("Skipping: Adapter not loaded.")

In [ ]:
if bbg_adapter and bbg_adapter.connected:
    print("Fetching option chain for SPX...")
    
    # This calls the get_option_chain method we modified
    instruments = bbg_adapter.get_option_chain(test_config[0])
    
    if len(instruments) > 0:
        print(f"✅ Success! Found {len(instruments)} contracts.")
        df = pd.DataFrame(instruments)
        display(df.head())
    else:
        print("⚠️ No instruments found. (Check if 'OPT_CHAIN' field is available for this ticker)")
else:
    print("Skipping: Not connected.")

In [ ]:
if bbg_adapter and bbg_adapter.connected:
    # 1. Define a test ticker (Use standard APP format)
    # This will be converted to "SPY US 12/20/25 C500 Equity" internally
    test_ticker = "SPY-27FEB26-690-C" 
    
    print(f"Subscribing to {test_ticker}...")
    
    # 2. Subscribe
    # The manager usually handles this, but we call adapter directly for testing
    bbg_adapter.subscribe([test_ticker])
    
    # 3. Watch for updates
    print("Waiting for data (10 seconds)...")
    start_time = time.time()
    
    while time.time() - start_time < 10:
        # Check the internal manager storage
        data = manager._tickers.get(test_ticker)
        if data:
            print(f"⚡ DATA RECEIVED: {data['last_price']} (Bid: {data['best_bid_price']} / Ask: {data['best_ask_price']})")
            break
        time.sleep(1)
        
    if not manager._tickers.get(test_ticker):
        print("❌ No data received. Market might be closed or ticker invalid.")
    else:
        print("✅ Real-time feed is working!")

In [ ]:
if bbg_adapter:
    print("Stopping Bloomberg session...")
    bbg_adapter.stop()
    print("Session stopped.")